In [ ]:
from node2vec import Node2Vec
import networkx as nx
from gensim.models import KeyedVectors
import community
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
import multiprocessing
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
graph_train_pos_nodes = np.load('data/graph_train_pos_nodes.npy')
graph_train_pos_edges = np.load('data/graph_train_pos_edges.npy')
edge_test = np.load('data/edge_test.npy')
label_test = np.load('data/label_test.npy')
edge_val = np.load('data/edge_val.npy')
label_val = np.load('data/label_val.npy')
edge_train = np.load('data/edge_train.npy')
label_train = np.load('data/label_train.npy')
graph_train_pos = nx.Graph()
graph_train_pos.add_nodes_from(graph_train_pos_nodes)
graph_train_pos.add_edges_from(graph_train_pos_edges)

In [ ]:
partition = community.best_partition(graph_train_pos)
for x in graph_train_pos.nodes():
    graph_train_pos.nodes[x]['community'] = partition[x]

def calculate_edge_coef(graph_train_pos, edge_posneg):
    rai = np.fromiter((i[2] for i in nx.resource_allocation_index(graph_train_pos, edge_posneg)), dtype=float)
    jc = np.fromiter((i[2] for i in nx.jaccard_coefficient(graph_train_pos, edge_posneg)), dtype=float)
    pa = np.fromiter((i[2] for i in nx.preferential_attachment(graph_train_pos, edge_posneg)), dtype=float)

    cn_sh = np.fromiter((i[2] for i in nx.cn_soundarajan_hopcroft(graph_train_pos, edge_posneg)), dtype=float)
    ra_idx_sh = np.fromiter((i[2] for i in nx.ra_index_soundarajan_hopcroft(graph_train_pos, edge_posneg)), dtype=float)
    wic = np.fromiter((i[2] for i in nx.within_inter_cluster(graph_train_pos, edge_posneg)), dtype=float)

    X = np.vstack((rai, jc, pa, cn_sh, ra_idx_sh, wic)).T
    return X

In [ ]:
X_train = calculate_edge_coef(graph_train_pos, edge_train)
X_val = calculate_edge_coef(graph_train_pos, edge_val)
X_test = calculate_edge_coef(graph_train_pos, edge_test)

np.save('data/edge_test_coef',X_test)
np.save('data/edge_val_coef',X_val)
np.save('data/edge_train_coef',X_train)

In [2]:
node2vec = Node2Vec(graph_train_pos, workers=1)
embedding_model = node2vec.fit(window=10, min_count=1, batch_words=4)
embedding_model.wv.save_word2vec_format('data/graph_train_pos_embedding.emb')
graph_train_pos_embedding = KeyedVectors.load_word2vec_format('data/graph_train_pos_embedding.emb')

Computing transition probabilities:   0%|          | 0/37700 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████████████████████████████████████████████████| 10/10 [27:04<00:00, 162.49s/it]
